# Directe rede van specifieke leeftijdsgroepen

## 1. Omvormen leeftijd(scategorieën) van ingeladen characterlist

In [307]:
import pandas as pd

In [308]:
def get_norm_age(file, degree='C'):
    df = pd.read_excel(file)
    norm_age = []
    if degree not in ('A', 'B', 'C'):
        raise ValueError('degree must be A, B or C')

    if degree == 'A':
        for age, stage in zip(df['age'], df['life stage']):
            if isinstance(stage, str):
                norm_age.append(stage)
            else:
                norm_age.append(age)
    
    elif degree == 'B':
        for age, stage in zip(df['age'], df['life stage']):
            if isinstance(stage, str):
                if stage in ('twenties', 'thirties', 'earlyadult'):
                    norm_age.append('earlyadult')
                elif stage in ('forties', 'fifties', 'middleadult'):
                    norm_age.append('middleadult')
                elif stage in ('sixties', 'seventies', 'oldadult'):
                    norm_age.append('oldadult')
                elif stage in ('eighties', 'nineties', 'deepoldadult'):
                    norm_age.append('deepoldadult')
                else:
                    norm_age.append(stage) #voor behoud van infant, earlychild, middlechild, latechild, child, adolescent, adult, ageless, unborn en abmiguous
            else:
                if age <= 2:
                    norm_age.append('infant')
                elif age <= 5:
                    norm_age.append('earlychild')
                elif age <= 8:
                    norm_age.append('middlechild')
                elif age <= 11:
                    norm_age.append('latechild')
                elif age <= 19:
                    norm_age.append('adolescent')
                elif age <= 39:
                    norm_age.append('earlyadult')
                elif age <= 59:
                    norm_age.append('middleadult')
                elif age <= 79:
                    norm_age.append('oldadult')
                else:
                    norm_age.append('deepoldadult')

    elif degree == 'C':
        for age, stage in zip(df['age'], df['life stage']):
            if isinstance(stage, str):
                if stage in ('infant', 'earlychild', 'middlechild', 'latechild', 'child'):
                    norm_age.append('child')
                elif stage in ('earlyadult','middleadult', 'twenties', 'thirties', 'forties', 'fifties', 'adult'):
                    norm_age.append('adult')
                elif stage in ('oldadult', 'deepoldadult', 'sixties', 'seventies', 'eighties', 'nineties'):
                    norm_age.append('oldadult')
                else:
                    norm_age.append(stage) #voor behoud 'adolescent', 'ageless', 'unborn' en 'ambiguous'
            else:
                if age <= 11:
                    norm_age.append('child')
                elif age <= 19:
                    norm_age.append('adolescent')
                elif age <= 59:
                    norm_age.append('adult')
                else:
                    norm_age.append('oldadult')

    df['norm'] = norm_age
    return df

In [309]:
#df = get_norm_age('annotaties/jeugdboeken/hetvogeltjevanamsterdamcharlistLG.xlsx', 'C')
#df.head()

## Stap 2: groepen opstellen ter vergelijking

In [310]:
def make_groups(df, ages):
    names = set(df['id'][df['norm'].isin(ages)])
    
    id2norm = dict(zip(df['id'], df['norm']))
    
    return names, id2norm

In [311]:
age_a = ['child']
age_b = ['adult']

## Stap 3: directe rede uit de xml halen

In [312]:
import argparse
from lxml import etree
namespaces = {'tei':'http://www.tei-c.org/ns/1.0'}

In [313]:
def speech_extract(title, degree):
    speech = []
    xml_path = 'annotaties/jeugdboeken/' + title + '.xml'
    excel_path = 'annotaties/jeugdboeken/' + title + '.xlsx'
    book = etree.parse(xml_path)
    df = get_norm_age(excel_path, degree=degree)
    names, id2norm = make_groups(df, ages=age_a + age_b)
    tmp = ''
    
    for said_node in book.iterfind('.//tei:said', namespaces=namespaces):
        if 'direct' in said_node.attrib and said_node.attrib['direct'] == 'true':
            if 'who' in said_node.attrib:
                name = said_node.attrib['who'].lower()
                if name in names:
                    v = ' '.join(said_node.itertext()).strip()
                    tmp += v
                    speech.append((name, id2norm[name], v))
        elif 'ana' in said_node.attrib and said_node.attrib['ana'] == '#reported':
            if 'who' in said_node.attrib:
                name = said_node.attrib['who'].lower()
                if name in names:
                    v = ' '.join(said_node.itertext()).strip()
                    tmp += v
                    speech.append((name, id2norm[name], v))

#export speech per title to txt
    #with open(f'{title}_child_speech', 'w') as f:
        #f.write(tmp)
    speech = [(a, b, c, title) for a, b, c in speech if c.strip()]
    return speech

In [314]:
import glob
import os
speech = []
# ugly workaround, now: same xls loaded twice with different names
titles = sorted(glob.glob('annotaties/jeugdboeken/*.xml'))
for title in titles:
    title = os.path.basename(title).replace('.xml', '')
    print(title)
    speech_ = speech_extract(title, degree='C')
    speech.extend(speech_)

dereddervanafrikaLG
detranenknallenuitmijnkopVJ
dezwartestenenVJ
drieverschrikkelijkedagenLDG
eendvooreendVJ
eengatindegrensVJ
eenhoofdvolmacaroniVJ
grotemensendaarkanjebetersoepvankokenVJ
hetboekvanalledingenVJ
hetdochtertjevandewasvrouwLDG
hetgelukkomtalsdedonderAM
hetisfijnomertezijnIR
hetvogeltjevanamsterdamLG
ikbenpollekehoorLDG
krasseninhettafelbladVJ
metdepoppengooienVJ
metdewindmeenaardezeeLDG
olleLDG
opjekopindeprullenbakVJ
pappaiseenhondLG
tintoevalendekunstvanhetverdwalenLDG
tintoevalendekunstvanmadeliefLDG
tintoevalenhetgeheimvantweebeenseilandLDG
tintoevalindeonderwereldLDG
vooraltijdsamenamenLDG


## Stap 4: metadata

In [315]:
speech = pd.DataFrame(speech, columns=['speaker', 'agecat', 'speech', 'title'])
print(speech.shape)
speech

(22562, 4)


,speaker,agecat,speech,title
0,brandijn,adult,Aanschouwer zie deez’ Moor : zijn vel is zw...,dereddervanafrikaLG
1,capitein25,adult,St. George d’Elmina\t\n5 november 1742\nAan de...,dereddervanafrikaLG
2,capitein25,adult,‘DEN HAAG!’,dereddervanafrikaLG
3,brandijn,adult,"woudreus uit het Haagse Bos, pikzwart van bas...",dereddervanafrikaLG
4,brandijn,adult,‘Het lichaam en het bloed van onze Verlosser’,dereddervanafrikaLG
5,ouders,adult,"‘Als zij leren lezen en schrijven, leren zi...",dereddervanafrikaLG
6,vanrijk,adult,"want predikanten , zei hij , waren in Hollan...",dereddervanafrikaLG
7,vanrijk,adult,"‘O zwarte man , schudt dat malle pak toch van...",dereddervanafrikaLG
8,ouders,adult,hun jongens eerst moeten leren speerwerpen ...,dereddervanafrikaLG
9,capiteinearlyadult,adult,"‘Dit kind hier, in deze kist, ‘t is dood, wat ...",dereddervanafrikaLG


In [316]:
#export all speech to txt
#my_list = speech['speech'].tolist()
#with open(f'jeugd_{age_a}_speech.txt', 'w') as f:
 #   for item in my_list:
  #      f.write("%s\n" % item)

In [317]:
df_int = pd.read_excel('intended_reader.xlsx')
int_read = dict(zip(df_int['title'], df_int['intended']))

speech['intended'] = speech['title'].map(int_read)
speech

,speaker,agecat,speech,title,intended
0,brandijn,adult,Aanschouwer zie deez’ Moor : zijn vel is zw...,dereddervanafrikaLG,volw
1,capitein25,adult,St. George d’Elmina\t\n5 november 1742\nAan de...,dereddervanafrikaLG,volw
2,capitein25,adult,‘DEN HAAG!’,dereddervanafrikaLG,volw
3,brandijn,adult,"woudreus uit het Haagse Bos, pikzwart van bas...",dereddervanafrikaLG,volw
4,brandijn,adult,‘Het lichaam en het bloed van onze Verlosser’,dereddervanafrikaLG,volw
5,ouders,adult,"‘Als zij leren lezen en schrijven, leren zi...",dereddervanafrikaLG,volw
6,vanrijk,adult,"want predikanten , zei hij , waren in Hollan...",dereddervanafrikaLG,volw
7,vanrijk,adult,"‘O zwarte man , schudt dat malle pak toch van...",dereddervanafrikaLG,volw
8,ouders,adult,hun jongens eerst moeten leren speerwerpen ...,dereddervanafrikaLG,volw
9,capiteinearlyadult,adult,"‘Dit kind hier, in deze kist, ‘t is dood, wat ...",dereddervanafrikaLG,volw


In [318]:
for t in speech['speech']: #voor elke cel-inhoud/string van de kolom 'speech'
    if not t.strip(): #als de string enkel whitespace is
        raise ValueError('Empty speech!')

In [319]:
speech['binary'] = 'X'
speech['binary'][speech['intended'] == 'jeugd']  = 'A'
speech['binary'][speech['intended'] == 'volw'] = 'B'
speech['binary'].value_counts()
speech

,speaker,agecat,speech,title,intended,binary
0,brandijn,adult,Aanschouwer zie deez’ Moor : zijn vel is zw...,dereddervanafrikaLG,volw,B
1,capitein25,adult,St. George d’Elmina\t\n5 november 1742\nAan de...,dereddervanafrikaLG,volw,B
2,capitein25,adult,‘DEN HAAG!’,dereddervanafrikaLG,volw,B
3,brandijn,adult,"woudreus uit het Haagse Bos, pikzwart van bas...",dereddervanafrikaLG,volw,B
4,brandijn,adult,‘Het lichaam en het bloed van onze Verlosser’,dereddervanafrikaLG,volw,B
5,ouders,adult,"‘Als zij leren lezen en schrijven, leren zi...",dereddervanafrikaLG,volw,B
6,vanrijk,adult,"want predikanten , zei hij , waren in Hollan...",dereddervanafrikaLG,volw,B
7,vanrijk,adult,"‘O zwarte man , schudt dat malle pak toch van...",dereddervanafrikaLG,volw,B
8,ouders,adult,hun jongens eerst moeten leren speerwerpen ...,dereddervanafrikaLG,volw,B
9,capiteinearlyadult,adult,"‘Dit kind hier, in deze kist, ‘t is dood, wat ...",dereddervanafrikaLG,volw,B


In [320]:
import nltk
from nltk.tokenize import sent_tokenize

lenA = 0
lenB = 0
senA = 0
senB = 0
        
speechA = set(speech['speech'][speech['binary'] == 'A'])
for a in speechA:
    lenA += len(a.split())
    senA += len(sent_tokenize(a))

speechB = set(speech['speech'][speech['binary'] == 'B'])
for b in speechB:
    lenB += len(b.split())
    senB +=len(sent_tokenize(b))
    
print('word count group A:', lenA)
print('word count group B:', lenB)
print('sentence count group A:', senA)
print('sentence count group B:', senB)

word count group A: 148362
word count group B: 99861
sentence count group A: 23400
sentence count group B: 7996


In [338]:
speech.to_excel('all_speech.xlsx')

Duale kolom aanmaken om over twee categoriën uit te splitsen:

## Stap 5: scatterplot

In [339]:
import scattertext as st
from scattertext.WhitespaceNLP import whitespace_nlp
nlp = whitespace_nlp

In [340]:
corpus = st.CorpusFromPandas(speech, 
            category_col='binary', 
            text_col='speech',
            nlp=whitespace_nlp).build()

In [341]:
html = st.produce_scattertext_explorer(corpus,
            category='A',
            pmi_threshold_coefficient=1000000, #ridicously high, to avoid getting bigrams
            #category_name='-'.join(age_a + gender_a),
            #not_category_name='-'.join(age_b + gender_b),
            width_in_pixels=1000,
            metadata=speech['speaker'])

with open('age_norm.html', 'wb') as f:
    f.write(html.encode('utf-8'))

/anaconda3/envs/clean36/lib/python3.6/site-packages/scattertext/frequencyreaders/DefaultBackgroundFrequencies.py:30: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  names=['word', 'background'])
/anaconda3/envs/clean36/lib/python3.6/site-packages/scattertext/frequencyreaders/DefaultBackgroundFrequencies.py:30: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  names=['word', 'background'])
/anaconda3/envs/clean36/lib/python3.6/site-packages/scattertext/TermDocMatrix.py:890: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  corpus_unigram_freq = corpus_freq_df.ix[[term for term


# Topic model

In [342]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [343]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')

In [364]:
# Data preprocessing
df = pd.read_excel('all_speech.xlsx')

data = df.speech.values.tolist()
pprint(data[:3])

['Aanschouwer zie deez’  Moor :  zijn  vel is zwart: maar wit  zijn  ziel, '
 'daar Jezus zelf als Priester voor  hem  bidt.  hij  gaat Geloof, en Hoop, en '
 'Liefde aan Moren leren, Opdat zij, witgemaakt, met  hem  het Lam steeds '
 'eren.\n'
 '  Brandijn Ryser',
 'St. George d’Elmina\t\n'
 '5 november 1742\n'
 'Aan de heer   Brandijn Ryser \n'
 'Het spijt  mij , beste  vriend ,  je  te moeten schrijven dat het  mij  niet '
 'goed gaat.  mijn  hoofd beeft van de gedachten die daarbinnen rondschuimen. '
 'Het duizelt  me , want ze proberen zich allemaal tegelijk naar buiten te '
 'persen.  ik  kan zoveel kina slikken als  ik  wil, deze koorts zal er niet '
 'door worden getemperd. Het is of de goede God heeft besloten de wereld '
 'ondersteboven te keren, uitsluitend en alleen om  mij , net als Job, te '
 'verwarren en te beproeven.  ik  ben bang dat  je  geen woord van  mijn  '
 'geraaskal zult begrijpen als  ik  niet heel diep in  mijn  geheugen tast en  '
 'je  alle feiten, ook die

In [260]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuation

data_words = list(sent_to_words(data))

print(data_words[:1])

[['aanschouwer', 'zie', 'deez', 'moor', 'zijn', 'vel', 'is', 'zwart', 'maar', 'wit', 'zijn', 'ziel', 'daar', 'jezus', 'zelf', 'als', 'priester', 'voor', 'hem', 'bidt', 'hij', 'gaat', 'geloof', 'en', 'hoop', 'en', 'liefde', 'aan', 'moren', 'leren', 'opdat', 'zij', 'witgemaakt', 'met', 'hem', 'het', 'lam', 'steeds', 'eren', 'brandijn', 'ryser']]


In [250]:
# Functions for stopwords and bigrams
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [251]:
# Remove stop words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)
print(id2word)

# Create Corpus
texts = data_words_nostops # or: data_words_bigrams

Dictionary(14536 unique tokens: ['aanschouwer', 'bidt', 'brandijn', 'deez', 'eren']...)


In [253]:
# Term frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]]


In [254]:
# Human readable format of term-frequency
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aanschouwer', 1),
  ('bidt', 1),
  ('brandijn', 1),
  ('deez', 1),
  ('eren', 1),
  ('gaat', 1),
  ('geloof', 1),
  ('hoop', 1),
  ('jezus', 1),
  ('lam', 1),
  ('leren', 1),
  ('liefde', 1),
  ('moor', 1),
  ('moren', 1),
  ('opdat', 1),
  ('priester', 1),
  ('ryser', 1),
  ('steeds', 1),
  ('vel', 1),
  ('wit', 1),
  ('witgemaakt', 1),
  ('zie', 1),
  ('ziel', 1),
  ('zwart', 1)]]

In [256]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [257]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"nee" + 0.029*"kinderen" + 0.021*"dag" + 0.017*"geloof" + 0.016*"erg" '
  '+ 0.016*"komen" + 0.016*"grote" + 0.015*"soms" + 0.014*"pappa" + '
  '0.013*"word"'),
 (1,
  '0.056*"we" + 0.033*"wel" + 0.027*"niks" + 0.023*"jij" + 0.022*"weet" + '
  '0.022*"moeder" + 0.021*"polleke" + 0.020*"zeg" + 0.019*"nou" + '
  '0.019*"caro"'),
 (2,
  '0.060*"zei" + 0.029*"meester" + 0.025*"spiek" + 0.015*"vroeg" + 0.015*"ga" '
  '+ 0.013*"weer" + 0.013*"keek" + 0.011*"kom" + 0.009*"terug" + 0.009*"ogen"'),
 (3,
  '0.044*"zegt" + 0.034*"opa" + 0.028*"mimoen" + 0.020*"waarom" + '
  '0.019*"misschien" + 0.016*"even" + 0.016*"ie" + 0.016*"gaat" + '
  '0.015*"meneer" + 0.015*"heel"'),
 (4,
  '0.054*"oma" + 0.047*"goed" + 0.019*"twee" + 0.019*"beetje" + 0.015*"mensen" '
  '+ 0.014*"vraagt" + 0.012*"vraag" + 0.012*"morgen" + 0.011*"heet" + '
  '0.010*"drie"')]


In [258]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/envs/clean36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.321138  0.127071       1        1  29.532784
1      0.215870  0.258832       2        1  27.938393
3      0.051057 -0.193445       3        1  16.087683
4      0.025876 -0.107171       4        1  13.800871
0      0.028335 -0.085288       5        1  12.640272, topic_info=      Category         Freq        Term        Total  loglift  logprob
984    Default  2338.000000         zei  2338.000000  30.0000  30.0000
1698   Default  2070.000000          we  2070.000000  29.0000  29.0000
7051   Default   981.000000         oma   981.000000  28.0000  28.0000
590    Default   861.000000         nee   861.000000  27.0000  27.0000
983    Default   939.000000        zegt   939.000000  26.0000  26.0000
319    Default   863.000000        goed   863.000000  25.0000  25.0000
946    Default  1217.000000         wel  1217.000000  24.0000  24.0000
542    Default  1146.000000     meester  1146.000000  23.0000  23.0000
7052   Default   721.000000         opa   721.000000  22.0000  22.0000
2243   Default   977.000000        niks   977.000000  21.0000  21.0000
10666  Default   979.000000       spiek   979.000000  20.0000  20.0000
10673  Default   596.000000      mimoen   596.000000  19.0000  19.0000
409    Default   831.000000         jij   831.000000  18.0000  18.0000
941    Default   815.000000        weet   815.000000  17.0000  17.0000
567    Default   798.000000      moeder   798.000000  16.0000  16.0000
444    Default   484.000000    kinderen   484.000000  15.0000  15.0000
10635  Default   769.000000     polleke   769.000000  14.0000  14.0000
1732   Default   747.000000         zeg   747.000000  13.0000  13.0000
3131   Default   716.000000         nou   716.000000  12.0000  12.0000
10638  Default   717.000000        caro   717.000000  11.0000  11.0000
1205   Default   706.000000        gaan   706.000000  10.0000  10.0000
386    Default   651.000000        hoor   651.000000   9.0000   9.0000
925    Default   433.000000      waarom   433.000000   8.0000   8.0000
918    Default   625.000000        waar   625.000000   7.0000   7.0000
1685   Default   597.000000       vroeg   597.000000   6.0000   6.0000
563    Default   393.000000   misschien   393.000000   5.0000   5.0000
155    Default   346.000000         dag   346.000000   4.0000   4.0000
234    Default   585.000000          ga   585.000000   3.0000   3.0000
819    Default   338.000000        twee   338.000000   2.0000   2.0000
3612   Default   338.000000      beetje   338.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
155     Topic5   345.250366         dag   346.287781   2.0653  -3.8769
6       Topic5   278.947479      geloof   279.998810   2.0645  -4.0901
221     Topic5   271.281403         erg   272.335632   2.0644  -4.1180
456     Topic5   263.250885       komen   264.290192   2.0643  -4.1481
336     Topic5   258.344269       grote   259.407532   2.0642  -4.1669
1973    Topic5   249.360748        soms   250.428604   2.0640  -4.2023
6845    Topic5   228.263199       pappa   229.346970   2.0635  -4.2907
2054    Topic5   213.752045        word   214.809357   2.0633  -4.3563
41      Topic5   185.804550        alle   186.865204   2.0626  -4.4965
2903    Topic5   175.243607         bal   176.309631   2.0622  -4.5550
6912    Topic5   125.046402         mam   126.181442   2.0592  -4.8925
662     Topic5   112.759506        paar   113.819664   2.0589  -4.9959
1115    Topic5   109.264427     blijven   110.301559   2.0588  -5.0274
1324    Topic5   104.767052       jawel   105.807549   2.0584  -5.0694
7224    Topic5   108.406219  verdrietig   109.529785   2.0580  -5.0353
351     Topic5    95.967216        hard    97.024460   2.0573  -5.1572
964     Topic5    94.528740       witte    95.608635   2.0569  -5.1723
226     Topic5    83.536613        eten    84.568047   2.0560  -5.2959
889     Topic5    84.452217  